In [45]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.autograd import Variable
import pandas as pd

In [46]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


Import raw data from yahoo finance

In [47]:
tsla_raw = pd.read_csv('TSLA.csv')
print(tsla_raw.shape)

(253, 7)


In [48]:
tsla_raw.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-08-25,235.676666,238.990005,234.666672,237.066666,237.066666,37936686
1,2021-08-26,236.103333,238.466660,232.540039,233.720001,233.720001,39642876
2,2021-08-27,235.000000,238.333328,234.033371,237.306671,237.306671,41501289
3,2021-08-30,238.238327,243.666672,237.576660,243.636673,243.636673,55812660
4,2021-08-31,244.333328,246.796661,242.146667,245.240005,245.240005,62566308


Calculate daily returns

In [49]:
def add_daily_return(market_data):
    market_data["Daily Return"]  = market_data['Close'] - market_data['Open']

add_daily_return(tsla_raw)
tsla_raw.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return
0,2021-08-25,235.676666,238.990005,234.666672,237.066666,237.066666,37936686,1.390000
1,2021-08-26,236.103333,238.466660,232.540039,233.720001,233.720001,39642876,-2.383332
2,2021-08-27,235.000000,238.333328,234.033371,237.306671,237.306671,41501289,2.306671
3,2021-08-30,238.238327,243.666672,237.576660,243.636673,243.636673,55812660,5.398346
4,2021-08-31,244.333328,246.796661,242.146667,245.240005,245.240005,62566308,0.906677


Calculate moving average (MA) of daily returns

In [51]:
def add_moving_average(market_data):
    N_SUMMANDS = 25
    temp_vars = []

    # df = market_data
    for i in range(0,N_SUMMANDS):
        temp_var = "M_{0}".format(i)
        market_data[temp_var] = market_data["Daily Return"].shift(i)
        temp_vars.append(temp_var)

    market_data["MA"] = market_data[temp_vars].mean(axis=1)

    for i in range(0,N_SUMMANDS):
        temp_var = "M_{0}".format(i)
        market_data.drop(temp_var, axis = 1, inplace = True)

add_moving_average(tsla_raw)
tsla_raw.head()


,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,MA
0,2021-08-25,235.676666,238.990005,234.666672,237.066666,237.066666,37936686,1.390000,1.390000
1,2021-08-26,236.103333,238.466660,232.540039,233.720001,233.720001,39642876,-2.383332,-0.496666
2,2021-08-27,235.000000,238.333328,234.033371,237.306671,237.306671,41501289,2.306671,0.437780
3,2021-08-30,238.238327,243.666672,237.576660,243.636673,243.636673,55812660,5.398346,1.677921
4,2021-08-31,244.333328,246.796661,242.146667,245.240005,245.240005,62566308,0.906677,1.523672


Build feature space

In [ ]:
def to_features(market_data):
  # print(market_data)
  return market_data.max()
  # def f(x):
  #  print (x) 
  #  #sample function
  #  return x.max()

In [ ]:
tsla_raw = tsla_raw.loc[:, tsla_raw.columns!='Date']
X = tsla_raw.apply(to_features, axis=1)
print(X)

0      37936686.0
1      39642876.0
2      41501289.0
3      55812660.0
4      62566308.0
          ...    
248    61395387.0
249    55843347.0
250    63985044.0
251    57259716.0
252    52742375.0
Length: 253, dtype: float64


In [ ]:
x = Variable(torch.rand(1, 6), requires_grad=True)
b = a**2
c = b*2
d = c.mean()
e = c.sum()

In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(tsla, batch_size=batch_size)
# test_dataloader = DataLoader(test_data, batch_size=batch_size)




# for X in train_dataloader:
#     print(f"Shape of X [N, C, H, W]: {X.shape}")
#     # print(f"Shape of y: {y.shape} {y.dtype}")
#     break

(253, 7)
